In [1]:
import os
import pyspark

from pymongo import MongoClient

In [2]:
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/09 11:36:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
username = "vt2182"
password = "vt2182"
host = "mongo-csgy-6513-spring.db"
auth_database = "vt2182"

In [4]:
mongo_conn_str = f"mongodb://{username}:{password}@{host}/{auth_database}"
client = MongoClient(mongo_conn_str)

In [5]:
db = client[auth_database]
db

Database(MongoClient(host=['mongo-csgy-6513-spring.db:27017'], document_class=dict, tz_aware=False, connect=True), 'vt2182')

In [6]:
db.list_collection_names()

['yellow_tripdata_raw']

In [7]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# Get the collection
collection = db['yellow_tripdata_raw']

In [9]:
# Fetch data from MongoDB, excluding the _id field
data = list(collection.find({}, {'_id': 0}))

In [10]:
# Convert list of dictionaries to pandas DataFrame
pandas_df = pd.DataFrame(data)

In [11]:
# Convert pandas DataFrame to Spark DataFrame
df = spark.createDataFrame(pandas_df)

/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [12]:
# Count the number of rows
print("The data contains {} rows.".format(df.count()))

23/05/09 11:37:03 WARN TaskSetManager: Stage 0 contains a task of very large size (8505 KiB). The maximum recommended task size is 1000 KiB.


The data contains 120000 rows.


In [13]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [14]:
df.head()

23/05/09 11:37:10 WARN TaskSetManager: Stage 3 contains a task of very large size (8505 KiB). The maximum recommended task size is 1000 KiB.


Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2022, 12, 3, 19, 14, 47), tpep_dropoff_datetime=datetime.datetime(2022, 12, 3, 19, 31, 51), passenger_count=1.0, trip_distance=3.37, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=143, DOLocationID=152, payment_type=1, fare_amount=13.5, extra=0.5, mta_tax=0.5, tip_amount=3.46, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=20.76, congestion_surcharge=2.5, airport_fee=0.0, __index_level_0__=375653)

In [15]:
df.limit(5).toPandas()

23/05/09 11:37:10 WARN TaskSetManager: Stage 4 contains a task of very large size (8505 KiB). The maximum recommended task size is 1000 KiB.


/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,__index_level_0__
0,2,2022-12-03 19:14:47,2022-12-03 19:31:51,1.0,3.37,1.0,N,143,152,1,13.50,0.5,0.5,3.46,0.0,0.3,20.76,2.5,0.0,375653
1,2,2022-12-01 11:55:12,2022-12-01 12:18:01,6.0,1.00,1.0,N,100,230,2,13.50,1.0,0.5,0.00,0.0,0.3,17.80,2.5,0.0,71522
2,1,2022-12-12 15:54:24,2022-12-12 15:59:52,1.0,1.90,1.0,N,236,74,1,7.50,3.0,0.5,2.25,0.0,0.3,13.55,2.5,0.0,1402168
3,1,2022-12-20 05:34:23,2022-12-20 06:14:12,1.0,0.10,99.0,N,242,242,1,21.50,0.0,0.5,0.00,0.0,1.0,23.00,0.0,0.0,2262245
4,2,2022-12-23 22:03:11,2022-12-23 22:28:44,NaN,6.53,NaN,None,164,61,0,31.83,0.0,0.5,3.00,0.0,1.0,38.83,NaN,NaN,3383941


Given the schema of the data, here are some preprocessing steps:

1. Removing unnecessary columns
2. Handling missing values
3. Creating new features

## Removing unnecessary columns

If there are any columns that are not relevant to predicting taxi demand, we can remove them to simplify the data and speed up the computations. For example, the column __index_level_0__ might not be relevant.

In [16]:
# Remove unnecessary columns
columns_to_drop = ['__index_level_0__']
df = df.drop(*columns_to_drop)

In [17]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



## Handling missing values

If there are any missing values in the data, we can drop the rows or columns that contain them or need to fill them in, depending on how many there are and their importance.

In [18]:
# Check for missing values

from pyspark.sql.functions import isnan, when, count, col

# Check for missing values in numerical columns
numerical_columns = ['VendorID', 'passenger_count', 'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in numerical_columns]).show()

# Check for missing values in non-numerical columns
non_numerical_columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'store_and_fwd_flag']
df.select([count(when(col(c).isNull(), c)).alias(c) for c in non_numerical_columns]).show()

23/05/09 11:38:23 WARN TaskSetManager: Stage 5 contains a task of very large size (8505 KiB). The maximum recommended task size is 1000 KiB.


+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       0|           4168|            0|      4168|           0|           0|           0|          0|    0|      0|         0|           0|                    0|           0|                4168|       4168|
+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+--

+--------------------+---------------------+------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|store_and_fwd_flag|
+--------------------+---------------------+------------------+
|                   0|                    0|              4168|
+--------------------+---------------------+------------------+



In [19]:
# Drop rows with missing values
df = df.dropna()

In [20]:
df.count()

23/05/09 11:38:29 WARN TaskSetManager: Stage 11 contains a task of very large size (8505 KiB). The maximum recommended task size is 1000 KiB.


115832

Alternatively, you can fill missing values with specific values
- replace missing numerical values with the median
- replace missing categorical values with the mode

## Creating new features

We can create new features that might be more predictive of taxi demand than the existing ones. For example, we can extract the hour of day, day of week, or month from the `tpep_pickup_datetime` and `tpep_dropoff_datetime` columns, which might be important for predicting demand.

In [21]:
from pyspark.sql.functions import year, month, dayofmonth, hour, dayofweek

# Create new features
df = df.withColumn('pickup_year', year(df['tpep_pickup_datetime']))
df = df.withColumn('pickup_month', month(df['tpep_pickup_datetime']))
df = df.withColumn('pickup_day', dayofmonth(df['tpep_pickup_datetime']))
df = df.withColumn('pickup_hour', hour(df['tpep_pickup_datetime']))
df = df.withColumn('pickup_day_of_week', dayofweek(df['tpep_pickup_datetime']))

df = df.withColumn('dropoff_year', year(df['tpep_dropoff_datetime']))
df = df.withColumn('dropoff_month', month(df['tpep_dropoff_datetime']))
df = df.withColumn('dropoff_day', dayofmonth(df['tpep_dropoff_datetime']))
df = df.withColumn('dropoff_hour', hour(df['tpep_dropoff_datetime']))
df = df.withColumn('dropoff_day_of_week', dayofweek(df['tpep_dropoff_datetime']))

In [22]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- pickup_year: integer (nullable = true)
 |-- pickup_month: integer (nullable = true)
 |-- pickup_day: integer (nullable = true)
 |-- pi

In [23]:
df.limit(5).toPandas()

23/05/09 11:38:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/05/09 11:38:38 WARN TaskSetManager: Stage 14 contains a task of very large size (8505 KiB). The maximum recommended task size is 1000 KiB.


/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_day_of_week,dropoff_year,dropoff_month,dropoff_day,dropoff_hour,dropoff_day_of_week
0,2,2022-12-03 19:14:47,2022-12-03 19:31:51,1.0,3.37,1.0,N,143,152,1,...,2022,12,3,19,7,2022,12,3,19,7
1,2,2022-12-01 11:55:12,2022-12-01 12:18:01,6.0,1.00,1.0,N,100,230,2,...,2022,12,1,11,5,2022,12,1,12,5
2,1,2022-12-12 15:54:24,2022-12-12 15:59:52,1.0,1.90,1.0,N,236,74,1,...,2022,12,12,15,2,2022,12,12,15,2
3,1,2022-12-20 05:34:23,2022-12-20 06:14:12,1.0,0.10,99.0,N,242,242,1,...,2022,12,20,5,3,2022,12,20,6,3
4,2,2022-12-10 03:59:40,2022-12-10 04:07:27,1.0,1.99,1.0,N,24,236,1,...,2022,12,10,3,7,2022,12,10,4,7


In [24]:
# Get the collection
collection = db['yellow_tripdata']

In [25]:
# Convert the PySpark DataFrame to a Pandas DataFrame
pandas_df = df.toPandas()

23/05/09 11:38:42 WARN TaskSetManager: Stage 15 contains a task of very large size (8505 KiB). The maximum recommended task size is 1000 KiB.


/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [26]:
# Convert the Pandas DataFrame to a dictionary
data_dict = pandas_df.to_dict("records")

In [27]:
# Insert the dictionary into MongoDB
collection.insert_many(data_dict)